# The Design Impact of Multiple Dispatch
## As the core paradigm of Julia
[@StefanKarpinski](http://twitter.com/StefanKarpinski)
presented at Strange Loop on September 19, 2013

[Original notebok](https://gist.github.com/StefanKarpinski/b8fe9dbb36c1427b9f22#file-multiple_dispatch-ipynb)
Converted by [@perfectionatic](http://twitter.com/perfectionatic) May 2020 to be Julia 1.x compatible 

# What is Multiple Dispatch?

- **dynamic** — based on actual run-time type, not static type
- **mulitple** — based on all arguments, not just the receiver

Tends to be written as **function application**:

- `f(a,b,c)` ⟸ LIKE THIS
- `a.f(b,c)` ⟸ NOT THIS

## Multiple Dispatch ≠ Method Overloading

In Java or C++ you can provide these virtual methods:

    Parent this: f(Parent that)
    Parent this: f(Child  that)
    Child  this: f(Parent that)
    Child  this: f(Child  that)

Dispatched on `this` but not on `that`

- can dispatch on both using the **double dispatch** pattern

However, quoting [Double Dispatch is a Code Smell][1]:

> The presence of Double Dispatch generally means that each type in a hierarchy has special handling code within another hierarchy of types. This approach to representing variant behavior leads to code that is less resilient to future changes as well as being more difficult to extend.

Something smells, but it's not necessarily the double dispatch

- *double dispatch is only a code smell in single dispatch languages*

[1]: http://lostechies.com/derekgreer/2010/04/19/double-dispatch-is-a-code-smell/

# Multiple Dispatch in Julia

#### Basic dispatch:

In [1]:
f(a::Any, b) = "fallback"
f(a::Number, b::Number) = "a and b are both numbers"
f(a::Number, b) = "a is a number"
f(a, b::Number) = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 5 methods)

In [2]:
f(1.5,2)

"a and b are both numbers"

In [3]:
f(1,"bar")

"a is a number"

In [4]:
f(1,2)

"a and b are both integers"

In [5]:
f("foo",[1,2])

"fallback"

#### "Diagonal" dispatch:

In [6]:
f(a::T, b::T) where T<:Number = "a and b are both $(T)s"

f (generic function with 6 methods)

In [7]:
f(big(1.5),big(2.5))

"a and b are both BigFloats"

In [8]:
f(big(1),big(2)) #<== integer rule is more specific

"a and b are both integers"

In [9]:
f("foo","bar") #<== still doesn't apply to non-numbers

"fallback"

#### Varargs methods:

In [10]:
f(args::Number...) = "$(length(args))-ary heterogeneous call"
f(args::T...) where {T<:Number}= "$(length(args))-ary homogeneous call"

f (generic function with 8 methods)

In [11]:
f(1)

"1-ary homogeneous call"

In [12]:
f(1,2,3)

"3-ary homogeneous call"

In [13]:
f(1,1.5,2)

"3-ary heterogeneous call"

In [14]:
f(1,2) # <== previous 2-arg method is more specific

"a and b are both integers"

In [15]:
f("foo") #<== doesn't apply to non-numbers

MethodError: MethodError: no method matching f(::String)
Closest candidates are:
  f(::Any, !Matched::Number) at In[1]:4
  f(::Any, !Matched::Any) at In[1]:1
  f(!Matched::Integer, !Matched::Integer) at In[1]:5
  ...

# Multiple Dispatch in Ruby

Arithmetic operators:

    Number + Number    | String + String  | Array + Array
    Number - Number    | Time - Time      | Time - Number     | Array - Array
    Number * Number    | Array * Integer  | Array * String    | String * Integer
    Integer << Integer | String << String | String << Integer

Arrays, Hashes & Strings:

    (Array|Hash).fetch(index,default|block)
    (Array|Hash).new(object|block) | String.new(string)
    (Array|Hash)[int|range]        | String[int|range|regex|string]
    (Array|Hash)[int|range]=       | String[int|range|regex|string]=
    Array.slice(int|range)         | String.slice(int|range|regex|string)
    Array.slice!(int|range)        | String.slice!(int|range|regex|string)

Just Strings:

    String.index(string|int|regex)
    String.rindex(string|int|regex)
    String.sub(pattern,replacement|block)
    String.sub!(pattern,replacement|block)
    String.gsub(pattern,replacement|block)
    String.gsub!(pattern,replacement|block)

# Multiple Dispatch in English

Related meanings:

    "she goes (home|away)"     go(subj::Noun, where::PlaceAdverb)
    "it went (wrong|well)"     go(subj::Noun, how::MannerAdverb)

Default arguments:

    "go (home|away|well)"      go(adv::Adverb) = go(Person("addressee"), adv)
    "he goes"                  go(subj::Noun)  = go(subj, PlaceAdverb("somewhere"))
    "go"                       go()            = go(PlaceAdverb("somewhere"))

Fragment of type hierarchy:

    Person <: Noun
    PlaceAdverb <: Adverb
    MannerAdverb <: Adverb

# Multiple Dispatch is Object-Oriented

Not surprising that o.o. languages are emulating multiple dispatch

- convenient, natural way to exress complex polymorphic behaviors
- and it's all about dispatch and subtyping of data

But they're missing a really crucial ingredient

- a bit like having `eval` without being able to manipulate code as data

# Generic Functions are Functional

A generic function is a **first-class object**

- can be passed around, e.g. to higher-order functions
- but remains extensible unlike "classical" functions

In [16]:
f

f (generic function with 8 methods)

In [17]:
methods(f)

# 8 methods for generic function "f":
[1] f(a::Integer, b::Integer) in Main at In[1]:5
[2] f(a::T, b::T) where T<:Number in Main at In[6]:1
[3] f(a::Number, b::Number) in Main at In[1]:2
[4] f(args::T...) where T<:Number in Main at In[10]:2
[5] f(args::Number...) in Main at In[10]:1
[6] f(a::Number, b) in Main at In[1]:3
[7] f(a, b::Number) in Main at In[1]:4
[8] f(a, b) in Main at In[1]:1

In [18]:
f2(x) = f(x,x)

f2("foo")

"fallback"

In [19]:
f(a::String, b::String) = "a and b are both strings";

f2("foo")

"a and b are both strings"

# The Expression Problem

Mads Torgersen's paper [The Expression Problem Revisited][2]:

> To which degree can your application be structured in such a way that both the data model and the set of virtual operations over it can be extended without the need to modify existing code, without the need for code repetition and without runtime type errors.

[2]: http://www.daimi.au.dk/~madst/ecoop04/main.pdf

Basically you want to be able to add:

1. **new types** to which you can apply existing operations → easy in o.o., hard in functional
2. **new operations** which you can apply to existing types → easy in function, hard in o.o.

# Interval Arithmetic

In [20]:
struct Interval{T<:Real} <: Number
  lo::T
  hi::T
end

(a::Real)..(b::Real) = Interval(a,b)

Base.show(io::IO, iv::Interval) = print(io, "($(iv.lo))..($(iv.hi))")

In [21]:
1..2

(1)..(2)

In [22]:
typeof(ans)

Interval{Int64}

In [23]:
sizeof(1..2) # ==> two 8-byte ints

16

In [24]:
(1.5)..(2.5)

(1.5)..(2.5)

In [25]:
typeof(ans)

Interval{Float64}

In [26]:
(1//2)..(2//3)

(1//2)..(2//3)

In [27]:
typeof(ans)

Interval{Rational{Int64}}

In [28]:
sizeof((1//2)..(2//3)) # ==> just four 8-byte ints

32

#### Allowing end-points of different types

In [29]:
methods(Interval)

# 4 methods for type constructor:
[1] (::Type{Interval})(lo::T, hi::T) where T<:Real in Main at In[20]:2
[2] (::Type{T})(x::T) where T<:Number in Core at boot.jl:715
[3] (::Type{T})(x::Base.TwicePrecision) where T<:Number in Base at twiceprecision.jl:243
[4] (::Type{T})(x::AbstractChar) where T<:Union{AbstractChar, Number} in Base at char.jl:50

In [30]:
#(0.5)..(1) #<== would be a no method error because of mixed types

In [31]:
Interval(lo::Real, hi::Real) = Interval(promote(lo,hi)...)

Interval

In [32]:
(0.5)..1

(0.5)..(1.0)

In [33]:
1..π

(1.0)..(3.141592653589793)

In [34]:
ℯ..π

(2.718281828459045)..(3.141592653589793)

#### Extending the `+` and `-` operators

Adding and subtracting intervals:

In [35]:
import Base:+,-

In [36]:
a::Interval + b::Interval = (a.lo + b.lo)..(a.hi + b.hi)
a::Interval - b::Interval = (a.lo - b.hi)..(a.hi - b.lo)

- (generic function with 175 methods)

In [37]:
(2..3) + (-1..1)

(1)..(4)

In [38]:
(2..3) - (1..2)

(0)..(2)

What about arithmetic mixing intervals and numbers?

- It's tempting to implement start implementing all combinations
- However, there's a better, more "Julian" way.

Instead, we'll hook `Intervals` into Julia's promotion system:

In [69]:
import Base: convert, promote_rule #<== allows extending them

convert(::Type{Interval{T}}, x::Real) where {T<:Real}= (x = convert(T,x); x..x)
convert(::Type{Interval{T}}, iv::Interval) where {T<:Real}=
    convert(T,iv.lo)..convert(T,iv.hi)

promote_rule(::Type{Interval{A}}, ::Type{B}) where {A<:Real,B<:Real}= 
    Interval{promote_type(A,B)}
promote_rule(::Type{Interval{A}}, ::Type{Interval{B}}) where {A<:Real,B<:Real}=
    Interval{promote_type(A,B)}

promote_rule (generic function with 124 methods)

Presto! Everything now Just Works™:

In [40]:
1..2 + 1

(1)..(3)

In [41]:
1..2 + 1.5

(1.0)..(3.5)

In [42]:
3 - 1..2 + 2//3

(2//1)..(8//3)

In [43]:
big(2)^100 + (1//3)..(2//3)

(3802951800684688204490109616129//3)..(2//3)

#### Wait? What's going on here?

Let's examine the last computation in detail:

In [44]:
big(2)^100 + (1//3)..(2//3)

(3802951800684688204490109616129//3)..(2//3)

In [45]:
typeof(ans)

Interval{Rational{BigInt}}

In [46]:
@which +(big(2)^100 ,(1//3)..(2//3))

+(x::Number, y::Number) in Base at promotion.jl:311

The left-hand-side is a `BigInt`:

In [47]:
big(2)^100

1267650600228229401496703205376

In [48]:
typeof(ans)

BigInt

The right-hand-side is an interval of rationals of 64-bit integers:

In [49]:
typeof((1//3)..(2//3))

Interval{Rational{Int64}}

There are no methods for adding these specific types

- a very general fallback method defined for adding numbers is called

This method is defined in `base/promote.jl`:

    +(x::Number, y::Number) = +(promote(x,y)...)

The `promote` function for two arguments looks like this:

    promote(x::T, y::S) where {T,S} =
        (convert(promote_type(T,S),x), convert(promote_type(T,S),y))

where `promote_type` is defined in terms of `promote_rule` — it's a bit complicated.

Now, the following promotion rules exist:

```
promote_rule(::Type{Interval{A}}, ::Type{B}) where {A<:Real,B<:Real} = 
    Interval{promote_type(A,B)}

promote_rule(::Type{Rational{A}}, ::Type{B}) where {A<:Integer,B<:Integer} =
    Rational{promote_type(A,B)}

promote_rule(::Type{BigInt}, ::Type{T}) where {T<:Integer}= BigInt
```

These all kick in when computing `promote_type`, producing this result:

In [50]:
promote_type(Interval{Rational{Int64}}, BigInt)

Interval{Rational{BigInt}}

So `promote` converts both `big(2)^100` and `(1//3)..(2//3)` to `Interval{Rational{Int64}}`:

In [51]:
convert(Interval{Rational{BigInt}}, big(2)^100)

(1267650600228229401496703205376//1)..(1267650600228229401496703205376//1)

In [52]:
convert(Interval{Rational{BigInt}}, (1//3)..(2//3))

(1//3)..(2//3)

After that adding them is just a call to the `+` method for same-typed `Intervals`.

#### Defining a new ± operator

The syntax for ± already exists in the parser, but its behavior is undefined:

In [53]:
:(2 ± 1)

:(2 ± 1)

In [54]:
2 ± 1

UndefVarError: UndefVarError: ± not defined

Let's define it:

In [55]:
a::Real     ± b::Real     = (a - b)..(a + b)
a::Interval ± b::Interval = (a.lo - b.hi)..(a.hi + b.hi)
a::Number   ± b::Number   = ±(promote(a,b)...)

± (generic function with 3 methods)

In [56]:
1 ± 2

(-1)..(3)

In [57]:
float(π) - (2 ± 1//2)

(0.6415926535897931)..(1.6415926535897931)

Notice that:

- hooking into the promotion system requires a bit of thought
- but it's a one-time cost – adding new promoting operations is easy
- types can work together smoothly without knowing about each other

## Interval Arithmetic Recap

#### Basic type definition

    immutable Interval <: Number where {T<:Real}
      lo::T
      hi::T
    end
    Interval(lo::Real, hi::Real) = Interval(promote(lo,hi)...)
    
    (a::Real)..(b::Real) = Interval(a,b)
    
    Base.show(io::IO, iv::Interval) = print(io, "($(iv.lo))..($(iv.hi))")
    
#### Conversion & promotion rules

    import Base: convert, promote_rule
    
    convert(::Type{Interval{T}}, x::Real) where {T<:Real}= (x = convert(T,x); x..x)
    convert(::Type{Interval{T}}, iv::Interval) where {T<:Real} =
        convert(T,iv.lo)..convert(T,iv.hi)
    
    promote_rule(::Type{Interval{A}}, ::Type{B}) where {A<:Real,B<:Real} = 
        Interval{promote_type(A,B)}
    promote_rule(::Type{Interval{A}}, ::Type{Interval{B}}) where {A<:Real,B<:Real}=
        Interval{promote_type(A,B)}
    
#### Core arithmetic

    a::Interval + b::Interval = (a.lo + b.lo)..(a.hi + b.hi)
    a::Interval - b::Interval = (a.lo - b.hi)..(a.hi - b.lo)
    
    a::Real     ± b::Real     = (a - b)..(a + b)
    a::Interval ± b::Interval = (a.lo - b.hi)..(a.hi + b.hi)
    a::Number   ± b::Number   = ±(promote(a,b)...)

Generic functions elegantly and smoothly solve the expression problem:

  - new `Interval` type works with built-in `+` and `-` operators
  - new `±` operator works with built-in `Int` type

# Taking Multiple Dispatch Seriously

Generic function in Julia aren't special – they're the default

- even really basic things like `+` are generic functions

In [58]:
+

+ (generic function with 167 methods)

This means you're free to extend *everything*

- not just functions that were planned to be extended

# Performance

In [59]:
code_llvm(+,(Int,Int))


;  @ int.jl:53 within `+'
define i64 @"julia_+_16728"(i64, i64) {
top:
  %2 = add i64 %1, %0
  ret i64 %2
}


In [60]:
code_llvm(+,(Int,Float64))


;  @ promotion.jl:311 within `+'
define double @"julia_+_17958"(i64, double) {
top:
; ┌ @ promotion.jl:282 within `promote'
; │┌ @ promotion.jl:259 within `_promote'
; ││┌ @ number.jl:7 within `convert'
; │││┌ @ float.jl:60 within `Float64'
      %2 = sitofp i64 %0 to double
; └└└└
;  @ promotion.jl:311 within `+' @ float.jl:401
  %3 = fadd double %2, %1
;  @ promotion.jl:311 within `+'
  ret double %3
}


## User Code is Also Fast

Let's define a scalar-valued function using the `Interval` type we just defined:

In [61]:
lo(a,b) = (a ± b).lo
hi(a,b) = (a ± b).hi

hi (generic function with 1 method)

In [62]:
lo(3,2), hi(3,2)

(1, 5)

In [63]:
code_llvm(lo,(Int,Int))


;  @ In[61]:1 within `lo'
define i64 @julia_lo_18155(i64, i64) {
top:
; ┌ @ In[55]:1 within `±'
; │┌ @ int.jl:52 within `-'
    %2 = sub i64 %0, %1
; └└
  ret i64 %2
}


In [64]:
code_llvm(hi,(Int,Int))


;  @ In[61]:2 within `hi'
define i64 @julia_hi_18156(i64, i64) {
top:
; ┌ @ In[55]:1 within `±'
; │┌ @ int.jl:53 within `+'
    %2 = add i64 %1, %0
; └└
  ret i64 %2
}


Even for complex combinations of types, generated code is very efficient:

In [65]:
code_native(hi,(Float64,Rational{Int}))

	.text
; ┌ @ In[61]:2 within `hi'
; │┌ @ In[55]:1 within `±'
; ││┌ @ promotion.jl:311 within `+'
; │││┌ @ promotion.jl:282 within `promote'
; ││││┌ @ promotion.jl:259 within `_promote'
; │││││┌ @ number.jl:7 within `convert'
; ││││││┌ @ rational.jl:100 within `AbstractFloat'
; │││││││┌ @ number.jl:7 within `convert'
; ││││││││┌ @ In[61]:2 within `Float64'
	vcvtsi2sdq	(%rdi), %xmm1, %xmm1
	vcvtsi2sdq	8(%rdi), %xmm2, %xmm2
; │││││││└└
; │││││││┌ @ float.jl:407 within `/'
	vdivsd	%xmm2, %xmm1, %xmm1
; │││└└└└└
; │││ @ promotion.jl:311 within `+' @ float.jl:401
	vaddsd	%xmm0, %xmm1, %xmm0
; │└└
	retq
	nopw	%cs:(%rax,%rax)
	nop
; └


# Design Impact

Since generic functions are **open**:

- functions are more like *protocols* which users can also implement

We're forced to think much harder about the **meaning** of opearations

- can't just describe their behavior.

Example:

> `search(string, pattern, [start])`
> 
> Search for the first occurance of the given pattern within the given string (or byte array). The pattern may be a single character, a vector or a set of characters, a string, or a regular expression (regular expressions are only allowed on contiguous strings, such as ASCII or UTF-8 strings). The third argument optionally specifies a starting index. The return value is a range of indexes where the matching sequence is found, such that `s[search(s,x)] == x`. The return value when the pattern is not found is `0` when the pattern is a character, or `0:-1` when searching for a substring or regular expression match.

# Meaning is Hard but Powerful

The `split` function is defined generically in terms of the `search` abstraction:

```
function split(str::String, splitter, limit::Integer, keep_empty::Bool)
    strs = String[]
    i = start(str)
    n = endof(str)
    r = search(str,splitter,i)
    j, k = first(r), nextind(str,last(r))
    while 0 < j <= n && length(strs) != limit-1
        if i < k
            if keep_empty || i < j
                push!(strs, str[i:prevind(str,j)])
            end
            i = k
        end
        if k <= j; k = nextind(str,j) end
        r = search(str,splitter,k)
        j, k = first(r), nextind(str,last(r))
    end
    if keep_empty || !done(str,i)
        push!(strs, str[i:])
    end
    return strs
end
split(s::String, spl, n::Integer) = split(s, spl, n, true)
split(s::String, spl, keep::Bool) = split(s, spl, 0, keep)
split(s::String, spl)             = split(s, spl, 0, true)
```

This gives the ability to:

- create a new pattern type for matching patterns in strings
  - e.g. context-free grammars, etc.
- define `search(str, pattern)` for the new pattern
  - get `split` functionality for free

### Example: spliting on regular expressions

Regex doesn't know anything about `split`:

In [66]:
methods(split)

# 4 methods for generic function "split":
[1] split(str::AbstractString; limit, keepempty) in Base at strings/util.jl:351
[2] split(str::T, splitter::Union{Tuple{Vararg{AbstractChar,N} where N}, Set{#s63} where #s63<:AbstractChar, AbstractArray{#s64,1} where #s64<:AbstractChar}; limit, keepempty) where T<:AbstractString in Base at strings/util.jl:317
[3] split(str::T, splitter::AbstractChar; limit, keepempty) where T<:AbstractString in Base at strings/util.jl:321
[4] split(str::T, splitter; limit, keepempty) where T<:AbstractString in Base at strings/util.jl:313

Yet split works when with `Regex` patterns:

In [67]:
split("foobarbazqux", r"[aeiou]+"i)

5-element Array{SubString{String},1}:
 "f"
 "b"
 "rb"
 "zq"
 "x"

Works because `Regex` provides the `search` function

- and `split` is defined generically in terms of `search`

### *Editor note* @perfectionatic... we are pretty much there now in (2020)
## We're Not There Yet

Meaning is hard and there are a lot of "protocols" that we still need to abstract out.

- There are separate `plot` functions in every plotting package
- Ideally, there should be a single commong `plot` function

These kinds of abstract protocols are "narrow waists"

- like TCP/IP in networking or byte streams in UNIX

Figuring out the right ones is hard but essential work.